# Text Prompts for Large Language Models (LLMs)

In this example, we will use the OpenAI API to interface with an LLM. 

## Set up the environment

OpenAI API access requires an API key (from [here](https://platform.openai.com)). Make sure to top up your balance before executing the queries to the LLM.

Alternatively, some open-source LLMs now follow the same OpenAI API, which you can run locally using programs such as [LM Studio](https://lmstudio.ai/).

Set your OpenAI API key:

In [ ]:
import openai

openai.api_key = 'your-api-key-here'

Define LLM properties:

In [ ]:
model_name = 'gpt-4'
temperature = 0.5
top_p = 0.
rng_seed = -1

# Testing the LLM

You can define a "system" prompt, which will determine the base behaviour of the LLM.

In [ ]:
system_prompt = "You are a helpful assistant that always replies in rhymes."

You can then send a single message to the LLM, and get a response back:

In [ ]:
from openai.chat.completions import create

user_message = 'What is the purpose of life?'

messages = {
	{"role": "system", "content": system_prompt},
    {"role": "user", "content": user_message}
}

output = create(model=model_name,
                temperature=temperature,
                top_p=top_p,
                messages=messages,
                seed=rng_seed
                ).choices[0].message

print(output.content)

To keep a conversation history, simply append the assistant response to `messages` and loop until the user wants to quit:

In [ ]:
messages = {
	{"role": "system", "content": system_prompt}
}

while True:
    user_message = input('You> ')
    messages.append({"role": "user", "content": user_message})
    output = create(model=model_name,
					temperature=temperature,
					top_p=top_p,
					messages=messages,
					seed=rng_seed
					).choices[0].message
    print('AI> ', output.content)
    messages.append({"role": "assistant", "content": output.content})

# Example Application

LLM responses can be used directly as genotypes for different kinds of content, or define changes to be applied to existing content.

In this simple example, we will use in-context learning to let the LLM update a string according to user-defined rules until a certain condition is met.

In [ ]:
def update_with_llm(x: str,
                    rules) -> str:
    rules_str = '\n'.join([f'{i}. Replace \'{a}\' with \'{b}\'' for i, (a, b) in enumerate(rules)])
    messages = [
		{'role': 'system', 'content': "Update the string provided by the users according to the following rules:\n"
									  f"{rules_str}\n"
                    				  "Only return the updated string."},
		{'role': 'user', 'content': x}
	]
    output = create(model=model_name,
					temperature=temperature,
					top_p=top_p,
					messages=messages,
					seed=rng_seed
					).choices[0].message
    return output.content

def is_terminated(x: str) -> bool:
    return 'c' in x

def process_and_log(eval_str, rules):
    iters = 0
    while not is_terminated(eval_str):
        eval_str = update_with_llm(eval_str)
        iters += 1
    print(f'Obtained \'{eval_str}\' after {iters} iterations.')


In [ ]:
starting_string = 'aaaaabababaaababa'
rules = [
	('aba', 'bbb')
	('bbb', 'c')
]

process_and_log(starting_string, rules)